In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import json
from datetime import datetime 
from alpaca_trade_api.rest import REST, TimeFrame
import time
from time import sleep
import datetime
import csv
%matplotlib inline

In [2]:
# Load .env enviroment variables
load_dotenv()
opensea_api_key = os.getenv("opensea_api_key")
opensea_api_key

'9796bf59fe674cae9688a1a9bb422338'

In [3]:
opensea_collections_url = "https://api.opensea.io/api/v1/collections"
opensea_collection_stats_url = "https://api.opensea.io/api/v1/collection/doodles-official/stats"

In [40]:
def get_events(url,start_date, end_date, event_type, collection_slug,limit=1):
    

    headers = {
        "Accept": "application/json",
        "X-API-KEY": opensea_api_key
    }
    
    query = { 
        "only_opensea": "true", 
        "occurred_before": end_date,
        "occurred_after": start_date,
        "event_type":event_type,
        "collection_slug":collection_slug,
        "limit":limit
    }
    
    response_data = requests.request("GET", url, headers=headers, params=query)
    json_data = response_data.json()
    #print(json_data)
    return json_data

In [43]:
#get_opensea_api_data(opensea_collections_url)
#get_opensea_api_data(opensea_collection_stats_url)

start_date = datetime.datetime(2022, 1, 10)
end_date = datetime.datetime(2022, 4, 11)
event_url = "https://api.opensea.io/api/v1/events";
event_type = "successful"
collection_slug = "boredapeyachtclub"
limit =1
response_data = get_events(event_url,
                                     start_date,
                                     end_date,
                                     event_type,
                                     collection_slug,
                                     limit)

print(response_data)
#print(response_data['asset_events'])


{'next': 'LWV2ZW50X3RpbWVzdGFtcD0yMDIyLTA0LTEwKzIwJTNBMzklM0EyMyYtcGs9NDg5NDIwMzg1Mg==', 'previous': None, 'asset_events': [{'approved_account': None, 'asset': {'id': 23137208, 'num_sales': 2, 'background_color': None, 'image_url': 'https://lh3.googleusercontent.com/GwkWWzNbZluPpHVhJwRpHs2rC_AuclHLpOFlObxi1kMcGlt33USynxmWAZyK2lfAElZGY1-CXAA7rimjPI90RBrlz-UHExsGUYSc', 'image_preview_url': 'https://lh3.googleusercontent.com/GwkWWzNbZluPpHVhJwRpHs2rC_AuclHLpOFlObxi1kMcGlt33USynxmWAZyK2lfAElZGY1-CXAA7rimjPI90RBrlz-UHExsGUYSc=s250', 'image_thumbnail_url': 'https://lh3.googleusercontent.com/GwkWWzNbZluPpHVhJwRpHs2rC_AuclHLpOFlObxi1kMcGlt33USynxmWAZyK2lfAElZGY1-CXAA7rimjPI90RBrlz-UHExsGUYSc=s128', 'image_original_url': 'ipfs://QmWrRYYYotLt4y3m7uJpWFLJdCLmrxZ5FPTTz5eB9tqMM2', 'animation_url': None, 'animation_original_url': None, 'name': None, 'description': None, 'external_link': None, 'asset_contract': {'address': '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d', 'asset_contract_type': 'non-fung

In [71]:
def parse_event(event):
    record = {}
    asset = event.get('asset')
    if asset == None:
        return None # if there's no asset that means it's not a single NFT transaction so skip this item

    #collection
    record['collection_slug'] = asset['collection']['slug']
    record['collection_name'] = asset['collection']['name']
    record['collection_url'] = "https://opensea.io/collection/" + asset['collection']['slug']

    #asset
    record['asset_id'] = asset['id']
    record['asset_name'] = asset['name']
    record['asset_description'] = asset['description']
    record['asset_contract_date'] = asset['asset_contract']['created_date']
    record['asset_url'] = asset['permalink']
    record['asset_img_url'] = asset['image_url']

    #event
    record['event_id'] = event['id']
    record['event_time'] = event.get('created_date')
    record['event_auction_type'] = event.get('auction_type')
    record['event_contract_address'] = event.get('contract_address')
    record['event_quantity'] = event.get('quantity')
    record['event_payment_symbol'] =  None if event.get('payment_token') == None else event.get('payment_token').get('symbol')

    decimals = 18
    if event.get('payment_token') != None:
        decimals = event.get('payment_token').get('decimals')

    price_str = event['total_price']

    try: 
        if len(price_str) < decimals:
            price_str =  "0." + (decimals-len(price_str)) * "0" + price_str
            record['event_total_price'] = float(price_str)
        else:
            record['event_total_price'] = float(price_str[:-decimals] + "." + price_str[len(price_str)-decimals:])
    except:
        print(event)

    return record

In [56]:
pause = 1

def fetch_all_events(url,start_date, end_date,event_type,collection_slug,limit):
    result = list()
    print(f"Fetching events between {start_date} and {end_date}")
    response = get_events(url,start_date,end_date,event_type,collection_slug, limit)
    for event in response['asset_events']:
        cleaned_event = parse_event(event)
        if cleaned_event != None:
            result.append(cleaned_event)
    return result

In [82]:


start_date = datetime.datetime(2021, 2, 10)
end_date = datetime.datetime(2022, 4, 10)
limit = 300

result = fetch_all_events(event_url,start_date,end_date,event_type,collection_slug, limit)

print(f"total number of records found:{len(result)}")

Fetching events between 2021-02-10 00:00:00 and 2022-04-10 00:00:00
total number of records found:300


In [75]:
file_name="event_data.csv"

#print(result[0].keys())

def write_csv(data, file_name):
    with open(file_name, mode='w', encoding='utf-8', newline='\n') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames = data[0].keys())

        writer.writeheader()
        for event in data:
            writer.writerow(event)
            
write_csv(result, file_name)

In [104]:
nft_events_df = pd.read_csv('event_data.csv')


nft_event_price_df=nft_events_df[['asset_id','event_time','event_total_price']]

nft_event_price_df['event_time']= pd.to_datetime(nft_events_df['event_time']).dt.date
nft_event_price_df.rename(columns = {'event_time':'event_date'}, inplace = True)

#nft_event_price_df =nft_event_price_df.set_index("event_date")
nft_event_price_df

/Users/sameerlakhe/opt/anaconda3/envs/alpacaenv/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/sameerlakhe/opt/anaconda3/envs/alpacaenv/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,asset_id,event_date,event_total_price
0,23137208,2022-04-09,101.6789
1,23131468,2022-04-09,130.0000
2,23123790,2022-04-09,110.0000
3,23134568,2022-04-09,107.0000
4,23133985,2022-04-09,125.0000
...,...,...,...
295,23132695,2022-03-18,94.0000
296,23142856,2022-03-18,94.0000
297,23140938,2022-03-18,100.0000
298,23133575,2022-03-18,90.5000


In [111]:
# this NFT collection has different assets which will have different prices based on various attributes 
# like rarity. For the sake of simplicity, we will consider the min/floor price of any of the asset
# for a particular day to determine the floor price of the collection for that day
# this function will interate though all the rows and check for different prices for a day.
# it will then pick the min of the price as the floor price for that day (irrespective of the assetId)
def consolidate_price_data_for_dates(df):
    prices_per_day={}
    for i in range(0, len(df)):
        event_date = df.iloc[i]['event_date']
        event_total_price = df.iloc[i]['event_total_price']
        #print(event_date,event_total_price )
        if event_date in prices_per_day:
            value_event_date = prices_per_day.get(event_date)
            value_event_date.append(event_total_price)
        else:
            prices_per_day[event_date]= [event_total_price]

    return prices_per_day

prices_per_day = consolidate_price_data_for_dates(nft_event_price_df)
prices_per_day

{datetime.date(2022, 4, 9): [101.6789, 130.0, 110.0, 107.0, 125.0],
 datetime.date(2022, 4, 8): [113.0, 114.0, 110.0, 130.0, 101.001, 124.0],
 datetime.date(2022, 4, 7): [125.0, 126.0, 123.99, 135.0, 129.0, 111.9],
 datetime.date(2022, 4, 6): [100.1775,
  117.0,
  112.0,
  122.0,
  129.4,
  112.0,
  124.68],
 datetime.date(2022, 4, 5): [119.69,
  120.0,
  124.54,
  111.0,
  122.388,
  133.0,
  111.0,
  118.0,
  98.85910100000004,
  180.0],
 datetime.date(2022, 4, 4): [111.0,
  118.0,
  109.99,
  109.69,
  108.9,
  100.0,
  96.85920303000005,
  120.0,
  110.0],
 datetime.date(2022, 4, 3): [105.0, 114.95, 124.0, 112.0, 125.0, 115.0],
 datetime.date(2022, 4, 2): [103.0, 200.0, 135.0, 133.333],
 datetime.date(2022, 4, 1): [102.0,
  115.0,
  111.69,
  125.0,
  121.95,
  121.5,
  111.0,
  118.5,
  125.0,
  100.0],
 datetime.date(2022, 3, 31): [125.0,
  119.5,
  114.69,
  117.9,
  115.0,
  159.0,
  115.24596696990398,
  108.5,
  110.0,
  124.5,
  108.9,
  109.0,
  140.0,
  200.0],
 datetime.d

In [127]:
# once the dictonary is created with key as date and values as different prices for that date,
# iterate over the dictionary, to get the lowest value for that date and create another dictionary

def get_floor_price_data(prices_per_day):
    floor_prices={}
    for key in prices_per_day:
        prices_for_current_day = prices_per_day[key]
        floor_price_current_day = min(prices_for_current_day)
        floor_prices[key]= floor_price_current_day
        print(key, 'corresponds to', floor_prices[key],floor_price_current_day)
    return floor_prices

floor_prices = get_floor_price_data(prices_per_day)
floor_prices

2022-04-09 corresponds to 101.6789 101.6789
2022-04-08 corresponds to 101.001 101.001
2022-04-07 corresponds to 111.9 111.9
2022-04-06 corresponds to 100.1775 100.1775
2022-04-05 corresponds to 98.85910100000004 98.85910100000004
2022-04-04 corresponds to 96.85920303000005 96.85920303000005
2022-04-03 corresponds to 105.0 105.0
2022-04-02 corresponds to 103.0 103.0
2022-04-01 corresponds to 100.0 100.0
2022-03-31 corresponds to 108.5 108.5
2022-03-30 corresponds to 102.69 102.69
2022-03-29 corresponds to 93.91 93.91
2022-03-28 corresponds to 0.029 0.029
2022-03-27 corresponds to 96.5 96.5
2022-03-26 corresponds to 95.01 95.01
2022-03-25 corresponds to 101.0 101.0
2022-03-24 corresponds to 97.0 97.0
2022-03-23 corresponds to 99.416835 99.416835
2022-03-22 corresponds to 100.69 100.69
2022-03-21 corresponds to 87.7334 87.7334
2022-03-20 corresponds to 85.0 85.0
2022-03-19 corresponds to 88.0 88.0
2022-03-18 corresponds to 90.5 90.5


{datetime.date(2022, 4, 9): 101.6789,
 datetime.date(2022, 4, 8): 101.001,
 datetime.date(2022, 4, 7): 111.9,
 datetime.date(2022, 4, 6): 100.1775,
 datetime.date(2022, 4, 5): 98.85910100000004,
 datetime.date(2022, 4, 4): 96.85920303000005,
 datetime.date(2022, 4, 3): 105.0,
 datetime.date(2022, 4, 2): 103.0,
 datetime.date(2022, 4, 1): 100.0,
 datetime.date(2022, 3, 31): 108.5,
 datetime.date(2022, 3, 30): 102.69,
 datetime.date(2022, 3, 29): 93.91,
 datetime.date(2022, 3, 28): 0.029,
 datetime.date(2022, 3, 27): 96.5,
 datetime.date(2022, 3, 26): 95.01,
 datetime.date(2022, 3, 25): 101.0,
 datetime.date(2022, 3, 24): 97.0,
 datetime.date(2022, 3, 23): 99.416835,
 datetime.date(2022, 3, 22): 100.69,
 datetime.date(2022, 3, 21): 87.7334,
 datetime.date(2022, 3, 20): 85.0,
 datetime.date(2022, 3, 19): 88.0,
 datetime.date(2022, 3, 18): 90.5}

In [128]:
#df = pd.DataFrame.from_dict(floor_prices)
df =pd.DataFrame(floor_prices.items(), columns=['event_date', 'event_total_price'])
df

,event_date,event_total_price
0,2022-04-09,101.678900
1,2022-04-08,101.001000
2,2022-04-07,111.900000
3,2022-04-06,100.177500
4,2022-04-05,98.859101
5,2022-04-04,96.859203
6,2022-04-03,105.000000
7,2022-04-02,103.000000
8,2022-04-01,100.000000
9,2022-03-31,108.500000
